In [0]:
import dlt
from pyspark import pipelines as dp
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
import dlt
from pyspark.sql.functions import col, trim, regexp_replace, to_timestamp, date_format

def clean_string(col_name):
    return trim(regexp_replace(col(col_name), '[^A-Za-z0-9 ]', ''))

def clean_date(col_name):
    return to_date(col(col_name), "yyyy-MM-dd")

@dlt.view
def silver_patients_cdf_v2():
    df = spark.readStream.format("delta").option("readChangeFeed", "true").table("LIVE.bronze_patients_cdf")
    df = df.withColumn("patient_id", clean_string("Id"))
    df = df.withColumn("FirstName", clean_string("FIRST"))
    df = df.withColumn("LastName", clean_string("LAST"))
    df = df.withColumn("date_of_birth", clean_date("BIRTHDATE"))
    return df

@dlt.view
def silver_conditions_cdf_v2():
    df = spark.readStream.format("delta").option("readChangeFeed", "true").table("LIVE.bronze_conditions_cdf")
    df = df.withColumn("visit_id", clean_string("ENCOUNTER"))
    df = df.withColumn("DiagnosisCode", clean_string("CODE"))
    df = df.withColumn("DiagnosisDescription", clean_string("DESCRIPTION"))
    return df

@dlt.view
def silver_encounters_cdf_v2():
    df = spark.readStream.format("delta").option("readChangeFeed", "true").table("LIVE.bronze_encounters_cdf")
    df = df.withColumn("visit_id", clean_string("Id"))
    df = df.withColumn("patient_id", clean_string("PATIENT"))
    df = df.withColumn("doctor_id", clean_string("PROVIDER"))
    df = df.withColumn("visit_ts", to_timestamp(col("START")))
    df = df.withColumn("EncounterCode", clean_string("CODE"))
    df = df.withColumn("EncounterDescription", clean_string("DESCRIPTION"))
    df = df.withColumn("visit_date_key", date_format(col("visit_ts"), "yyyyMMdd").cast("int"))
    return df

@dlt.view
def silver_providers_cdf_v2():
    df = spark.readStream.format("delta").option("readChangeFeed", "true").table("LIVE.bronze_providers_cdf")
    df = df.withColumn("doctor_id", clean_string("Id"))
    df = df.withColumn("DoctorName", clean_string("NAME"))
    df = df.withColumn("Specialty", clean_string("SPECIALITY"))
    return df


In [0]:

dlt.create_streaming_table("patients_type2_stage")
dlt.apply_changes(
    target = "patients_type2_stage",
    source = "silver_patients_cdf_v2",
    keys = ["patient_id"],
    sequence_by ="_commit_timestamp",          
    ignore_null_updates = True,
    apply_as_deletes=expr("_change_type = 'delete'"),
    stored_as_scd_type = 2
)

dlt.create_streaming_table("conditions_type2_stage")
dlt.apply_changes(
    target="conditions_type2_stage",
    source="silver_conditions_cdf_v2",
    keys = ["visit_id", "DiagnosisCode"],      
    sequence_by ="_commit_timestamp",          
    apply_as_deletes=expr("_change_type = 'delete'"),
    ignore_null_updates=True,
    stored_as_scd_type= 2
)

dlt.create_streaming_table("encounters_type2_stage")
dlt.apply_changes(
    target="encounters_type2_stage",
    source="silver_encounters_cdf_v2",
    keys=["visit_id"],                    
    sequence_by ="_commit_timestamp",          
    apply_as_deletes=expr("_change_type = 'delete'"),
    ignore_null_updates=True,
    stored_as_scd_type= 2
)

dlt.create_streaming_table("providers_type2_stage")
dlt.apply_changes(
    target="providers_type2_stage",
    source="silver_providers_cdf_v2",
    keys=["doctor_id"],
    sequence_by ="_commit_timestamp",          
    apply_as_deletes=expr("_change_type = 'delete'"),
    ignore_null_updates=True,
    stored_as_scd_type= 2
)